
# Análisis RFM - Entrega 1

- **Definición del problema**
- **Diccionario de datos**
- **Script inicial para leer datos y preparar métricas RFM**
---


## 1. Definición del problema

El objetivo es **segmentar clientes** utilizando el método **RFM (Recency, Frequency, Monetary)** con base en un conjunto de transacciones históricas.

- **Recency (R)**: Días desde la última compra.
- **Frequency (F)**: Número total de compras.
- **Monetary (M)**: Monto total gastado.

El análisis permitirá clasificar a los clientes en diferentes segmentos  y generar recomendaciones personalizadas.

**Entradas**:
- Archivo CSV con transacciones de clientes.

**Salidas**:
- Tabla con métricas RFM por cliente.
- Visualizaciones básicas (histogramas, ranking).

---



## 2. Diccionario de datos

| Columna      | Tipo     | Descripción |
|--------------|----------|-------------|
| CustomerID   | String   | Identificador único del cliente (ej. C001) |
| InvoiceNo    | Integer  | Número de factura o transacción |
| InvoiceDate  | Date     | Fecha de la transacción (YYYY-MM-DD) |
| Amount       | Float    | Valor monetario de la transacción |

---


##3. Script inicial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ruta del archivo CSV (ajusta si es necesario)
csv_file = "transactions_rfm.csv"

# Cargar datos
df = pd.read_csv(csv_file)

# Convertir columna InvoiceDate a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Mostrar primeras filas y estructura
print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
print(df.head())

In [ ]:
# Preparar cálculo RFM
# Fecha de referencia (fecha actual)
fecha_referencia = pd.Timestamp.today()

# 1. Calcular la fecha de la última compra por cliente
last_purchase = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase['Recency'] = (fecha_referencia - last_purchase['InvoiceDate']).dt.days

print("=== Última compra y Recency ===")
print(last_purchase.head(), "\n")

In [ ]:
# 2. Calcular la frecuencia (cantidad de facturas por cliente)
frequency = df.groupby('CustomerID')['InvoiceNo'].count().reset_index()
frequency.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)

print("=== Frecuencia por cliente ===")
print(frequency.head(), "\n")

In [ ]:
# 3. Calcular el valor monetario (suma de Amount por cliente)
monetary = df.groupby('CustomerID')['Amount'].sum().reset_index()
monetary.rename(columns={'Amount': 'Monetary'}, inplace=True)

print("=== Valor Monetario por cliente ===")
print(monetary.head(), "\n")

In [ ]:
# 4. Unir todo en un solo DataFrame
rfm = last_purchase[['CustomerID', 'Recency']].merge(frequency, on='CustomerID').merge(monetary, on='CustomerID')

print("=== Matriz RFM final ===")
print(rfm.head())

In [ ]:
# Histograma Recency
plt.figure(figsize=(8, 5))
plt.hist(rfm['Recency'], bins=15, color='#1f77b4', edgecolor='black')
plt.title('Distribución de Recency')
plt.xlabel('Días desde última compra')
plt.ylabel('Número de clientes')
plt.show()

# Histograma Frequency
plt.figure(figsize=(8, 5))
plt.hist(rfm['Frequency'], bins=15, color='#ff7f0e', edgecolor='black')
plt.title('Distribución de Frequency')
plt.xlabel('Número de compras')
plt.ylabel('Número de clientes')
plt.show()

# Histograma Monetary
plt.figure(figsize=(8, 5))
plt.hist(rfm['Monetary'], bins=15, color='#2ca02c', edgecolor='black')
plt.title('Distribución de Monetary')
plt.xlabel('Monto total gastado')
plt.ylabel('Número de clientes')
plt.show()

# Ranking de usuarios que mas han gastado
top_10 = rfm.sort_values(by='Monetary', ascending=False).head(10)

plt.figure(figsize=(8, 5))
plt.bar(top_10['CustomerID'], top_10['Monetary'], color='#9467bd', edgecolor='black')
plt.title('Top 10 clientes por gasto total')
plt.xlabel('CustomerID')
plt.ylabel('Monto total')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Mostrar también el ranking en tabla
print("=== Top 10 clientes por valor monetario ===")
print(top_10)


---
#Analisis RFM - Entrega 2

##Novedades
- **Nuevas columnas y variables**
- **Propuesta de ML**
- **Dataset de mejor calidad**
- **Script mejorado**
---

##1. Nuevas columnas y variables

###Columnas

- **cutomer_id**: Identificador del comprador.
- **order_id**: Identificador de la factura/transaccion.
- **order_date**: Fecha de la factura/transaccion.
- **total_amopunt**: Total de la compra.
- **category (columna categórica)**:Categoria de la compra (suponiendo que una factura/transaccion solo tiene una categoria).
- **returned**: Si el producto de devolvio o no.
- **profit_margin**: Margen de ganancia por la compra.
<br>

###Variables calculadas

- **RFM**
  - **Recency**: fecha actual – última compra (días).
  - **Frequency**: número de facturas por cliente.
  - **Monetary**: gasto total.
<br>

- **Derivadas de gasto**
  - **Ticket promedio**: Monetary / Frequency.
  - **Velocidad de gasto**: Monetary / Recency.
<br>

- **Derivadas de tiempo**
  - **Antigüedad del cliente**: fecha actual – primera compra (días).
  - **Porcentaje de meses activos**: meses con ≥1 compra / meses totales desde alta.
<br>

- **Variables de lealtad**
  - **Ratio de repetición**: (Frequency – 1) / Frequency.
<br>

- **Variables de categorias**
  - **Producto favorito**: categoría más frecuente por cliente.
  - **Concentración de gasto (Herfindahl)** = ∑(pᵢ²), con pᵢ = % gasto en categoría i.
<br>

- **Variables de calidad de cliente**
  - **Porcentaje de devoluciones**:  pedidos devueltos / total pedidos.
  - **Margen medio por cliente**: promedio de profit_margin sobre sus compras.

---



##2. Propuesta ML

- **K-Means**: Asume que los datos se pueden dividir en k grupos, cada uno alrededor de un centroide (punto medio). Itera hasta que cada punto está más cerca de un centroide que de los demás.<br>Se usara como guia base.
<br>

- **GMM (Gaussian Mixture Models)**: Supone que los datos son una mezcla de distribuciones gaussianas (campanas de Gauss). Usa EM (Expectation-Maximization) para ajustar esas distribuciones y asignar probabilidades a cada punto de pertenecer a cada cluster. <br> Se realizan clusters mas realistas y maneja mejor los datos sesgados.
<br>

- **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**: Agrupa puntos según densidad. Define dos parámetros:

  - **eps**: radio máximo de vecindad.

  - **min_samples**: número mínimo de puntos para considerar un “núcleo”.

  - Crea clusters si hay suficiente densidad; los puntos que no encajan son valores atipicos (outliers).

  Podria ayudar a detectar clientes VIP ya que no son tan comunes en un ecommerce.
  
---

##3. Dataset

Este data set es ficticio y representa ventas de un ecommerce, tiene 34.500 filas y 17 columnas. Fue diseñado cuidadosamente para simular de manera realista las ventas de un tienda virtual

Resumen de columnas

- order_id: Identificador único de cada pedido
- customer_id: Identificador único de cada cliente
- product_id: Identificador único de cada producto
- category: Categoría del producto (Electrónica, Moda, Hogar, - Belleza, Deportes, Juguetes, Abarrotes)
- price: Precio unitario del producto
- discount: Descuento aplicado (%)
- quantity: Número de artículos comprados
- payment_method: Tipo de pago (Tarjeta de Crédito, Tarjeta de - Débito, UPI, PayPal, Contraentrega, Billetera)
- order_date: Fecha de la compra
- delivery_time_days: Días que tardó en entregarse el pedido
- region: Región geográfica del cliente
- returned: Si el producto fue devuelto (Sí/No)
- total_amount: Monto final de la factura después de descuentos
- shipping_cost: Costos de envío
- profit_margin: Ganancia obtenida del pedido
- customer_age: Edad del cliente (18–70)

[Link Dataset Kaggle](https://www.kaggle.com/datasets/miadul/e-commerce-sales-transactions-dataset?resource=download)

---

##4. Script mejorado
---

###4.1. Calculo de RFM y otras metricas


- Genera métricas adicionales: ticket promedio, velocidad de - gasto, antigüedad, porcentaje de meses activos, ratio de repetición.
- Usa categorías para producto favorito y concentración de gasto.
- Deriva porcentaje de devoluciones y margen medio.
- Aplica log-transformación a variables sesgadas.
- Escala todas las numéricas con StandardScaler.
- Codifica producto_favorito con One-Hot.
- Devuelve un dataset listo (sales_rfm_final) para clustering con - K-Means, GMM o DBSCAN
---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# ==========================
# 1. Cargar dataset
# ==========================
sales_df = pd.read_csv("orders.csv", parse_dates=["order_date"])
print("\nPrimeras filas:")
print(sales_df.head())

fecha_ref = pd.Timestamp.today()

In [ ]:
# ==========================
# 2. Variables RFM
# ==========================
sales_rfm = sales_df.groupby("customer_id").agg(
    last_purchase=("order_date", "max"),
    first_purchase=("order_date", "min"),
    Frequency=("order_id", "nunique"),
    Monetary=("total_amount", "sum")
).reset_index()

sales_rfm["Recency"] = (fecha_ref - sales_rfm["last_purchase"]).dt.days
sales_rfm["Antiguedad"] = (fecha_ref - sales_rfm["first_purchase"]).dt.days

sales_rfm.drop(columns=["last_purchase", "first_purchase"], inplace=True)


In [ ]:
# ===============================
# 3. Variables derivadas de gasto
# ===============================
sales_rfm["Ticket_promedio"] = sales_rfm["Monetary"] / sales_rfm["Frequency"]
sales_rfm["Velocidad_gasto"] = sales_rfm["Monetary"] / (sales_rfm["Recency"] + 1)  # +1 evita división por cero

In [ ]:
# ==============================
# 4. Porcentaje de meses activos
# ==============================
sales_df["year_month"] = sales_df["order_date"].dt.to_period("M")

meses_activos = sales_df.groupby("customer_id")["year_month"].nunique().reset_index()
meses_activos.rename(columns={"year_month": "meses_activos"}, inplace=True)

meses_totales = (fecha_ref.to_period("M") - sales_df.groupby("customer_id")["order_date"].min().dt.to_period("M")).apply(lambda x: x.n).reset_index()
meses_totales.rename(columns={"order_date": "meses_totales"}, inplace=True)

sales_rfm = sales_rfm.merge(meses_activos, on="customer_id").merge(meses_totales, on="customer_id")
sales_rfm["pct_meses_activos"] = sales_rfm["meses_activos"] / (sales_rfm["meses_totales"] + 1)

In [ ]:
# ==========================
# 5. Ratio de repetición
# ==========================
sales_rfm["ratio_repeticion"] = (sales_rfm["Frequency"] - 1) / sales_rfm["Frequency"]

In [ ]:
# =============================================
# 6. Producto favorito y concentración de gasto
# =============================================
cat_gasto = sales_df.groupby(["customer_id", "category"])["total_amount"].sum().reset_index()

# Producto favorito
producto_fav = cat_gasto.loc[cat_gasto.groupby("customer_id")["total_amount"].idxmax()][["customer_id", "category"]]
producto_fav.rename(columns={"category": "producto_favorito"}, inplace=True)

# Concentración de gasto (Herfindahl)
cat_gasto["pct"] = cat_gasto.groupby("customer_id")["total_amount"].transform(lambda x: x / x.sum())
herfindahl = cat_gasto.groupby("customer_id")["pct"].apply(lambda x: (x**2).sum()).reset_index()
herfindahl.rename(columns={"pct": "concentracion_gasto"}, inplace=True)

sales_rfm = sales_rfm.merge(producto_fav, on="customer_id").merge(herfindahl, on="customer_id")

In [ ]:
# =============================
# 7. Porcentaje de devoluciones
# =============================
sales_df["returned_flag"] = sales_df["returned"].apply(lambda x: 1 if str(x).lower() == "yes" else 0)
devoluciones = sales_df.groupby("customer_id")["returned_flag"].mean().reset_index()
devoluciones.rename(columns={"returned_flag": "pct_devoluciones"}, inplace=True)

sales_rfm = sales_rfm.merge(devoluciones, on="customer_id")

In [ ]:
# ==========================
# 8. Margen medio
# ==========================
margen = sales_df.groupby("customer_id")["profit_margin"].mean().reset_index()
margen.rename(columns={"profit_margin": "margen_medio"}, inplace=True)

sales_rfm = sales_rfm.merge(margen, on="customer_id")

print("\nPrimeras filas:")
print(sales_rfm.head())

In [ ]:
# ==========================
# 9. Limpieza de datos
# ==========================

# Log-transformación en variables sesgadas
cols_log = ["Monetary", "Frequency", "Ticket_promedio", "Velocidad_gasto"]
for c in cols_log:
    sales_rfm[c] = np.log1p(sales_rfm[c])  # log(1+x) para evitar log(0)

# Escalar variables numéricas
num_cols = ["Recency", "Frequency", "Monetary", "Ticket_promedio",
            "Velocidad_gasto", "Antiguedad", "pct_meses_activos",
            "ratio_repeticion", "concentracion_gasto",
            "pct_devoluciones", "margen_medio"]

scaler = StandardScaler()
sales_rfm_scaled = scaler.fit_transform(sales_rfm[num_cols])

# One-Hot para producto favorito
sales_rfm_final = pd.get_dummies(sales_rfm[["customer_id", "producto_favorito"]], columns=["producto_favorito"])
sales_rfm_final = sales_rfm_final.merge(pd.DataFrame(sales_rfm_scaled, columns=num_cols), left_index=True, right_index=True)

In [ ]:
# ==========================
# Resultado
# ==========================
print("Dataset final listo para clustering:")
print(sales_rfm_final.head())

---
###4.2. Clustering

- **K-Means**: prueba de 2 a 9 clusters, elige el mejor según - Silhouette.
- **GMM**: igual, pero usando Gaussian Mixture, elige el mejor.
- **DBSCAN**: marca outliers (-1 = ruido).<br>
  Métricas:
  - **Silhouette**: más alto es mejor (cohesión vs separación).
  - **Davies-Bouldin**: más bajo es mejor.
- **Visualización**: reducción con PCA para ver clusters en 2D.
- **Resumen**: imprime métricas y distribución de clusters.
---

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns

# ===========================
# 1. Dataset para clustering
# ===========================
# Excluimos customer_id (es identificador, no feature)
X = sales_rfm_final.drop(columns=["customer_id"])

In [ ]:
# ==========================
# 2. K-Means
# ==========================
scores = {}
silhouettes = {}
davies = {}

# Probar varios k
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    scores[f"KMeans_{k}"] = labels
    silhouettes[f"KMeans_{k}"] = sil
    davies[f"KMeans_{k}"] = db

best_kmeans = max(silhouettes, key=silhouettes.get)
print("Mejor K-Means:", best_kmeans,
      "Silhouette:", silhouettes[best_kmeans],
      "DB:", davies[best_kmeans])

sales_rfm_final["Cluster_KMeans"] = scores[best_kmeans]

In [ ]:
# ===============================
# 3. Gaussian Mixture Model (GMM)
# ===============================
for k in range(2, 10):
    gmm = GaussianMixture(n_components=k, covariance_type="full", random_state=42)
    labels = gmm.fit_predict(X)
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    scores[f"GMM_{k}"] = labels
    silhouettes[f"GMM_{k}"] = sil
    davies[f"GMM_{k}"] = db

best_gmm = max(silhouettes, key=lambda k: silhouettes[k] if k.startswith("GMM") else -1)
print("Mejor GMM:", best_gmm,
      "Silhouette:", silhouettes[best_gmm],
      "DB:", davies[best_gmm])

sales_rfm_final["Cluster_GMM"] = scores[best_gmm]

In [ ]:
# ==========================
# 4. DBSCAN para outliers
# ==========================
dbscan = DBSCAN(eps=2, min_samples=5)  # Ajustar parámetros según dataset
labels_db = dbscan.fit_predict(X)
sales_rfm_final["Cluster_DBSCAN"] = labels_db

In [ ]:
# ==========================
# 5. Visualización PCA (2D)
# ==========================
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(18,5)) # Adjusted figure size for 3 plots

# KMeans plot
plt.subplot(1,3,1)
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=sales_rfm_final["Cluster_KMeans"], palette="tab10", s=50)
plt.title("Clusters K-Means (PCA)")

# GMM plot
plt.subplot(1,3,2)
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=sales_rfm_final["Cluster_GMM"], palette="tab10", s=50)
plt.title("Clusters GMM (PCA)")

# DBSCAN plot
plt.subplot(1,3,3)
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=sales_rfm_final["Cluster_DBSCAN"], palette="tab10", s=50)
plt.title("Clusters DBSCAN (PCA)") # Added title for DBSCAN plot

plt.tight_layout()
plt.show()

In [ ]:
# ==========================
# 6. Resultados finales
# ==========================
print("Resumen Silhouette Scores:")
for k, v in silhouettes.items():
    print(k, "→", round(v, 3))

print("\nResumen Davies-Bouldin Scores (menor es mejor):")
for k, v in davies.items():
    print(k, "→", round(v, 3))

print("\nDBSCAN encontró clusters (label -1 = outliers):")
print(pd.Series(labels_db).value_counts())

In [ ]:
# ==========================
# 7. Interpretación de clusters
# ==========================

# Variables de negocio a analizar por cluster
vars_negocio = [
    "Recency", "Frequency", "Monetary", "Ticket_promedio", "Velocidad_gasto",
    "Antiguedad", "pct_meses_activos", "ratio_repeticion",
    "concentracion_gasto", "pct_devoluciones", "margen_medio"
]

def resumir_clusters(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean().round(2)
    resumen["count_clientes"] = df.groupby(cluster_col).size()
    return resumen.sort_values("Monetary", ascending=False)

# Resumen KMeans
print("=== Perfil de Clusters K-Means ===")
print(resumir_clusters(sales_rfm_final, "Cluster_KMeans"))

# Resumen GMM
print("\n=== Perfil de Clusters GMM ===")
print(resumir_clusters(sales_rfm_final, "Cluster_GMM"))

# Resumen DBSCAN (solo para clusters, ignora -1 si quieres ver solo clientes válidos)
print("\n=== Perfil de Clusters DBSCAN ===")
print(resumir_clusters(sales_rfm_final, "Cluster_DBSCAN"))

---
###4.3. Etiquetas de los clusters
---

In [ ]:
def etiquetar_clusters(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean()
    etiquetas = {}

    for cluster, row in resumen.iterrows():
        if row["Monetary"] > resumen["Monetary"].median() and row["Frequency"] > resumen["Frequency"].median() and row["Recency"] < resumen["Recency"].median() and row["margen_medio"] > resumen["margen_medio"].median():
            etiquetas[cluster] = "VIP Rentables"

        elif row["Monetary"] > resumen["Monetary"].median() and row["margen_medio"] < resumen["margen_medio"].median():
            etiquetas[cluster] = "Big Spenders con bajo margen"

        elif row["Monetary"] > resumen["Monetary"].median() and row["margen_medio"] > resumen["margen_medio"].median():
            etiquetas[cluster] = "Big Spenders con alto margen"

        elif row["Velocidad_gasto"] > resumen["Velocidad_gasto"].quantile(0.75):
            etiquetas[cluster] = "Sprinters (alta velocidad de gasto)"

        elif row["Frequency"] > resumen["Frequency"].median() and row["pct_meses_activos"] > 0.5:
            etiquetas[cluster] = "Compradores constantes"

        elif row["ratio_repeticion"] < 0.5:
            etiquetas[cluster] = "Compradores únicos / transaccionales"

        elif row["pct_devoluciones"] > 0.2:
            etiquetas[cluster] = "Cazadores de ofertas / problemáticos"

        elif row["concentracion_gasto"] > 0.7:
            etiquetas[cluster] = "Especialistas de categoría"

        elif row["concentracion_gasto"] <= 0.7:
            etiquetas[cluster] = "Diversificados"

        else:
            etiquetas[cluster] = "Clientes duraderos pero de bajo valor"

    df[f"{cluster_col}_Etiqueta"] = df[cluster_col].map(etiquetas)
    return df, etiquetas


In [ ]:
# Etiquetar clusters KMeans
sales_rfm_final, etiquetas_kmeans = etiquetar_clusters(sales_rfm_final, "Cluster_KMeans")
print("Etiquetas KMeans:", etiquetas_kmeans)

# Etiquetar clusters GMM
sales_rfm_final, etiquetas_gmm = etiquetar_clusters(sales_rfm_final, "Cluster_GMM")
print("Etiquetas GMM:", etiquetas_gmm)

# Etiquetar clusters DBSCAN
sales_rfm_final, etiquetas_dbscan = etiquetar_clusters(sales_rfm_final, "Cluster_DBSCAN")
print("Etiquetas DBSCAN:", etiquetas_dbscan)


---
###4.4. Representacion visual de los clusters
---


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================
# 1. Heatmap de promedios normalizados por cluster
# ==========================
def plot_heatmap(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean()
    # Normalizar cada columna 0–1 para comparabilidad
    resumen_norm = (resumen - resumen.min()) / (resumen.max() - resumen.min())

    plt.figure(figsize=(12,6))
    sns.heatmap(resumen_norm, annot=True, cmap="Blues", fmt=".2f", cbar=True)
    plt.title(f"Perfil normalizado de {cluster_col}", fontsize=14)
    plt.ylabel("Cluster")
    plt.show()

# ==========================
# 2. Boxplots por variable y cluster
# ==========================
def plot_boxplots(df, cluster_col, variables):
    n = len(variables)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))

    if n == 1:
        axes = [axes]

    for ax, var in zip(axes, variables):
        sns.boxplot(x=cluster_col, y=var, data=df, ax=ax, palette="Set3")
        ax.set_title(f"{var} por cluster")
        ax.set_xlabel("Cluster")
        ax.set_ylabel(var)

    plt.tight_layout()
    plt.show()

# ==========================
# 3. Ejecutar reportes
# ==========================
# Heatmap para KMeans
plot_heatmap(sales_rfm_final, "Cluster_KMeans_Etiqueta")

# Boxplots de variables clave para KMeans
plot_boxplots(sales_rfm_final, "Cluster_KMeans_Etiqueta", ["Recency", "Frequency", "Monetary", "Ticket_promedio", "pct_devoluciones"])


---
###4.5. Reporte de la clasificacion
---

In [ ]:
def generar_reporte_segmentos(df, cluster_col):
    resumen = df.groupby(cluster_col)[vars_negocio].mean().round(2)
    reporte = []

    for cluster, row in resumen.iterrows():
        desc = f"Segmento {cluster} ({cluster_col}):\n"

        # Recency
        if row["Recency"] < resumen["Recency"].median():
            desc += "- Clientes recientes (compraron hace poco).\n"
        else:
            desc += "- Clientes inactivos (última compra hace tiempo).\n"

        # Frequency
        if row["Frequency"] > resumen["Frequency"].median():
            desc += "- Compran con frecuencia.\n"
        else:
            desc += "- Compran rara vez.\n"

        # Monetary
        if row["Monetary"] > resumen["Monetary"].median():
            desc += "- Gastan por encima del promedio.\n"
        else:
            desc += "- Gastan por debajo del promedio.\n"

        # Ticket promedio
        if row["Ticket_promedio"] > resumen["Ticket_promedio"].median():
            desc += "- Ticket promedio alto (compras grandes).\n"
        else:
            desc += "- Ticket promedio bajo (compras pequeñas).\n"

        # Velocidad de gasto
        if row["Velocidad_gasto"] > resumen["Velocidad_gasto"].median():
            desc += "- Alta velocidad de gasto (aportan rápido).\n"

        # % meses activos
        if row["pct_meses_activos"] > 0.5:
            desc += "- Se mantienen activos la mayor parte del tiempo.\n"
        else:
            desc += "- Activos en pocos meses.\n"

        # Ratio repetición
        if row["ratio_repeticion"] < 0.5:
            desc += "- Mayoría son compradores de una sola vez.\n"

        # % devoluciones
        if row["pct_devoluciones"] > 0.2:
            desc += "- Alta tasa de devoluciones, clientes problemáticos.\n"
        else:
            desc += "- Baja tasa de devoluciones.\n"

        # Margen
        if row["margen_medio"] > resumen["margen_medio"].median():
            desc += "- Compran productos de buen margen.\n"
        else:
            desc += "- Suelen comprar con márgenes bajos.\n"

        # Concentración
        if row["concentracion_gasto"] > 0.7:
            desc += "- Especialistas: concentran gasto en pocas categorías.\n"
        else:
            desc += "- Diversificados: compran en varias categorías.\n"

        desc += "\n"
        reporte.append(desc)

    return "\n".join(reporte)

# Generar reporte para KMeans con etiquetas
print("=== Reporte narrativo KMeans ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_KMeans_Etiqueta"))

print("\n=== Reporte narrativo GMM ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_GMM_Etiqueta"))

print("\n=== Reporte narrativo DBSCAN ===\n")
print(generar_reporte_segmentos(sales_rfm_final, "Cluster_DBSCAN_Etiqueta"))
